# Ntpl 0 0 2_Vector
Write and read STL vectors with RNTuple.  Adapted from the hvector tree tutorial.




**Author:** The ROOT Team  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 04:05 AM.</small></i>

Note: the rntuple classes are experimental at this point.
 Functionality, interface, and data format is still subject to changes.
 Do not use for real data!

In [1]:
%%cpp -d
#include <ROOT/RNTuple.hxx>
#include <ROOT/RNTupleModel.hxx>

#include <TCanvas.h>
#include <TH1F.h>
#include <TRandom.h>
#include <TSystem.h>

#include <cstdio>
#include <iostream>
#include <memory>
#include <vector>
#include <utility>

input_line_26:1:10: fatal error: 'ROOT/RNTuple.hxx' file not found
#include <ROOT/RNTuple.hxx>
         ^~~~~~~~~~~~~~~~~~


Import classes from experimental namespace for the time being

In [2]:
using RNTupleModel = ROOT::Experimental::RNTupleModel;
using RNTupleReader = ROOT::Experimental::RNTupleReader;
using RNTupleWriter = ROOT::Experimental::RNTupleWriter;

input_line_27:1:42: error: no type named 'RNTupleModel' in namespace 'ROOT::Experimental'
using RNTupleModel = ROOT::Experimental::RNTupleModel;void __cling_Un1Qu32(void* vpClingValue) {
                     ~~~~~~~~~~~~~~~~~~~~^
input_line_27:3:43: error: no type named 'RNTupleReader' in namespace 'ROOT::Experimental'
using RNTupleReader = ROOT::Experimental::RNTupleReader;
                      ~~~~~~~~~~~~~~~~~~~~^
input_line_27:4:43: error: no type named 'RNTupleWriter' in namespace 'ROOT::Experimental'
using RNTupleWriter = ROOT::Experimental::RNTupleWriter;
                      ~~~~~~~~~~~~~~~~~~~~^


Where to store the ntuple of this example

In [3]:
constexpr char const* kNTupleFileName = "ntpl002_vector.root";

Update the histogram gui every so many fills

In [4]:
constexpr int kUpdateGuiFreq = 1000;

Number of events to generate

In [5]:
constexpr int kNEvents = 25000;

  Generate kNEvents with vectors in kNTupleFileName
 

In [6]:
%%cpp -d
void Write()
{
   // We create a unique pointer to an empty data model
   auto model = RNTupleModel::Create();

   // Creating fields of std::vector is the same as creating fields of simple types.  As a result, we get
   // shared pointers of the given type
   std::shared_ptr<std::vector<float>> fldVpx = model->MakeField<std::vector<float>>("vpx");
   auto fldVpy   = model->MakeField<std::vector<float>>("vpy");
   auto fldVpz   = model->MakeField<std::vector<float>>("vpz");
   auto fldVrand = model->MakeField<std::vector<float>>("vrand");

   // We hand-over the data model to a newly created ntuple of name "F", stored in kNTupleFileName
   // In return, we get a unique pointer to an ntuple that we can fill
   auto ntuple = RNTupleWriter::Recreate(std::move(model), "F", kNTupleFileName);

   TH1F hpx("hpx", "This is the px distribution", 100, -4, 4);
   hpx.SetFillColor(48);

   auto c1 = new TCanvas("c1", "Dynamic Filling Example", 200, 10, 700, 500);

   gRandom->SetSeed();
   for (int i = 0; i < kNEvents; i++) {
      int npx = static_cast<int>(gRandom->Rndm(1) * 15);

      fldVpx->clear();
      fldVpy->clear();
      fldVpz->clear();
      fldVrand->clear();

      // Set the field data for the current event
      for (int j = 0; j < npx; ++j) {
         float px, py, pz;
         gRandom->Rannor(px, py);
         pz = px*px + py*py;
         auto random = gRandom->Rndm(1);

         hpx.Fill(px);

         fldVpx->emplace_back(px);
         fldVpy->emplace_back(py);
         fldVpz->emplace_back(pz);
         fldVrand->emplace_back(random);
      }

      // Gui updates
      if (i && (i % kUpdateGuiFreq) == 0) {
         if (i == kUpdateGuiFreq) hpx.Draw();
         c1->Modified();
         c1->Update();
         if (gSystem->ProcessEvents())
            break;
      }

      ntuple->Fill();
   }

   hpx.DrawCopy();

   // The ntuple unique pointer goes out of scope here.  On destruction, the ntuple flushes unwritten data to disk
   // and closes the attached ROOT file.
}

input_line_35:4:17: error: use of undeclared identifier 'RNTupleModel'
   auto model = RNTupleModel::Create();
                ^
input_line_35:8:84: error: expected '(' for function-style cast or type construction
   std::shared_ptr<std::vector<float>> fldVpx = model->MakeField<std::vector<float>>("vpx");
                                                                 ~~~~~~~~~~~~~~~~~~^
input_line_35:9:55: error: expected '(' for function-style cast or type construction
   auto fldVpy   = model->MakeField<std::vector<float>>("vpy");
                                    ~~~~~~~~~~~~~~~~~~^
input_line_35:10:55: error: expected '(' for function-style cast or type construction
   auto fldVpz   = model->MakeField<std::vector<float>>("vpz");
                                    ~~~~~~~~~~~~~~~~~~^
input_line_35:11:55: error: expected '(' for function-style cast or type construction
   auto fldVrand = model->MakeField<std::vector<float>>("vrand");
                                    ~~~~~~~~~

  For all of the events, histogram only one of the written vectors
 

In [7]:
%%cpp -d
void Read()
{
   // Get a unique pointer to an empty RNTuple model
   auto model = RNTupleModel::Create();

   // We only define the fields that are needed for reading
   auto fldVpx = model->MakeField<std::vector<float>>("vpx");

   // Create an ntuple without imposing a specific data model.  We could generate the data model from the ntuple
   // but here we prefer the view because we only want to access a single field
   auto ntuple = RNTupleReader::Open(std::move(model), "F", kNTupleFileName);

   // Quick overview of the ntuple's key meta-data
   std::cout << ntuple->GetInfo();
   // In a future version of RNTuple, there will be support for ntuple->Show() and ntuple->Scan()

   TCanvas *c2 = new TCanvas("c2", "Dynamic Filling Example", 200, 10, 700, 500);
   TH1F h("h", "This is the px distribution", 100, -4, 4);
   h.SetFillColor(48);

   // Iterate through all the events using i as event number and as an index for accessing the view
   for (auto entryId : *ntuple) {
      ntuple->LoadEntry(entryId);

      for (auto px : *fldVpx) {
         h.Fill(px);
      }

      if (entryId && (entryId % kUpdateGuiFreq) == 0) {
         if (entryId == kUpdateGuiFreq) h.Draw();
         c2->Modified();
         c2->Update();
         if (gSystem->ProcessEvents())
            break;
      }
   }

   // Prevent the histogram from disappearing
   h.DrawCopy();
}

input_line_36:4:17: error: use of undeclared identifier 'RNTupleModel'
   auto model = RNTupleModel::Create();
                ^
input_line_36:7:53: error: expected '(' for function-style cast or type construction
   auto fldVpx = model->MakeField<std::vector<float>>("vpx");
                                  ~~~~~~~~~~~~~~~~~~^
input_line_36:11:18: error: use of undeclared identifier 'RNTupleReader'
   auto ntuple = RNTupleReader::Open(std::move(model), "F", kNTupleFileName);
                 ^


In [8]:
Write();
Read();

input_line_38:2:3: error: use of undeclared identifier 'Write'
 (Write())
  ^
Error in <HandleInterpreterException>: Error evaluating expression (Write()).
Execution of your code was aborted.


Draw all canvases 

In [9]:
gROOT->GetListOfCanvases()->Draw()